In [1]:
# ollama pull llama3.2
import json

from ollama import chat
from ollama import ChatResponse
import minsearch 

from elasticsearch import Elasticsearch
from tqdm.auto import tqdm

/home/cdrleo/Desktop/courses/alexey_llm_zoomcamp/llm_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import elasticsearch
print(elasticsearch.__version__)

(8, 18, 1)


In [3]:
def prepare_document(document_path: str):
    with open(document_path, "r") as f_in:
        raw_documents = json.load(f_in)

    documents = []

    for course_dict in raw_documents:
        for doc in course_dict["documents"]:
            doc["course"] = course_dict["course"]
            documents.append(doc)

    return documents

def search(documents: list, query: str, num_results: int = 5, filter_dict: dict = {"course": "data-engineering-zoomcamp"}):
    index = minsearch.Index(
        text_fields=["question", "text", "section"], 
        keyword_fields=["course"]
    )
    index.fit(documents)

    boost = {
        "question": 3.0, "section": 0.5,
        # "text": 1.0,  "course": 0.5
    }   

    results = index.search(
        query=query, 
        boost_dict=boost, 
        num_results=num_results,
        filter_dict=filter_dict
    )

    return results

In [4]:
def llm(question, results):
  
  prompt_template = """
  you're a course teaching assistant. answer the QUESTION based on the context below. 
  use only the facts in the context to answer the question. 
  if you don't know the answer, say "I don't know".

  QUESTION: {question}

  CONTEXT: {context}
  """

  context = "" 

  for doc in results:
      context = context + f"\nsection: {doc['section']}\n" + \
          f"question: {doc['question']}\n" + \
          f"text: {doc['text']}\n\n"
  prompt = prompt_template.format(question=question, context=context).strip()

  response: ChatResponse = chat(model="llama3.2", messages=[
    {
      "role": "user",
      "content": prompt,
    },
  ])

  return (response.message.content)

In [5]:
def rag(question: str, num_results: int = 5, filter_dict: dict = {"course": "data-engineering-zoomcamp"}):
    documents = prepare_document("documents.json")
    results = search(documents, question, num_results, filter_dict)
    
    if len(results) == 0:
        return "I don't know"

    answer = llm(question, results)

    return answer

In [6]:
# question = "the course already started, can I still join?"

# rag(question)

before importing ElasticSearch, run the server:

```
docker run -it \
    --rm \
    --name elasticsearch \
    -m 4GB \
    -p 9200:9200 \
    -p 9300:9300 \
    -e "discovery.type=single-node" \
    -e "xpack.security.enabled=false" \
    docker.elastic.co/elasticsearch/elasticsearch:8.4.3
```

In [7]:
import requests
print(requests.get("http://localhost:9200").text)

{
  "name" : "19fa9022a345",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "nZZEiSkoSmmSFaoEykAMag",
  "version" : {
    "number" : "8.4.3",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "42f05b9372a9a4a470db3b52817899b99a76ee73",
    "build_date" : "2022-10-04T07:17:24.662462378Z",
    "build_snapshot" : false,
    "lucene_version" : "9.3.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}



In [8]:
es_client = Elasticsearch('http://localhost:9200', request_timeout=60)

In [9]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': False, 'index': 'course-questions'})

In [10]:
es_client

<Elasticsearch(['http://localhost:9200'])>

In [11]:
documents = prepare_document("documents.json")

In [12]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [15]:
documents_len = len(documents)
documents_len

948

In [20]:
for document in tqdm(documents):
    es_client.index(index=index_name, document=document)

  0%|          | 0/948 [00:00<?, ?it/s]

In [ ]:
def elastic_search(query: str, size: int = 5):
    search_query = {
        "size": size,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs